## ENVIRONMENT SETUP

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
%%bash
rm ek_data -R
rm sample_data -R # remove pre-existing data from the storage

# As read and write operations from google drive are slow, we suggest to copy and unzip
# the dataset in a local directory on the Colab's machine.
mkdir -p ek_data/frames

for SLICE in "D1" # D1 "D2" "D3" "ActionNet"
do
  # Copy the *.tar.gz files of Epic-Kitchens
  cp /content/drive/MyDrive/MLDL23/EPIC-Kitchens-55/$SLICE/*.tar.gz ./ek_data

  # Untar
  for file in ./ek_data/*.tar.gz; do
    fn=$(basename $file)
    fn=${fn/.tar.gz/}
    # ls -lah $file
    mkdir -p ek_data/frames/$fn
    tar xf $file --directory=ek_data/frames/$fn
  done

  # Remove the archives already extracted
  rm ./ek_data/*.tar.gz
done

In [ ]:
!rm mldl23-ego/ -R

In [1]:
# Clone the public repository (or your repository)
# !git clone https://github.com/EgovisionPolito/mldl23-ego.git mldl23-ego
!git clone -b main https://www.github.com/antoniodalmaso/mldl23-ego.git mldl23-ego

Cloning into 'mldl23-ego'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 282 (delta 36), reused 51 (delta 18), pack-reused 211
Receiving objects: 100% (282/282), 343.94 MiB | 16.94 MiB/s, done.
Resolving deltas: 100% (135/135), done.
Updating files: 100% (72/72), done.


In [ ]:
# Importing the checkpoints (only of D1 for now)
import zipfile

!mkdir ./mldl23-ego/saved_models/
!cp /content/drive/MyDrive/MLDL23/I3D_SourceOnlyD1.zip ./mldl23-ego/saved_models/

with zipfile.ZipFile("./mldl23-ego/saved_models/I3D_SourceOnlyD1.zip", 'r') as zip_ref:
    zip_ref.extractall("./mldl23-ego/saved_models/")

!rm ./mldl23-ego/saved_models/I3D_SourceOnlyD1.zip

In [ ]:
# Install conda on Google Colab

!pip install -q condacolab
import condacolab

condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...


In [ ]:
%%bash
# Create a conda environment

# -----------------------------------------------------------------------
# conda env create --name egovision -f mldl23-ego/requirements.yaml

# conda install conda-pack
# conda pack -n egovision -o egovision.tar.gz -f --ignore-missing-files

# mv egovision.tar.gz /content/drive/MyDrive/MLDL23/
#------------------------------------------------------------------------

cp /content/drive/MyDrive/MLDL23/egovision.tar.gz ./
mkdir -p /usr/local/envs/egovision
tar -xzf egovision.tar.gz -C /usr/local/envs/egovision

rm egovision.tar.gz

## FEATURES EXTRACTION

In [ ]:
%%bash

cd mldl23-ego

PYTHON_PATH=/usr/local/envs/egovision/bin/python
$PYTHON_PATH save_feat.py name=D1 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames

## EPIC KITCHENS CLASSIFIER

In [ ]:
%%bash

cd mldl23-ego

PYTHON_PATH=/usr/local/envs/egovision/bin/python
$PYTHON_PATH script_EK.py \
  linear \
  /content/mldl23-ego/saved_features/last_D1_train.pkl \
  /content/mldl23-ego/saved_features/last_D1_test.pkl \
  0.01 \
  0.9 \
  40

## ACTIONET RGB EXPERIMENTS

In [ ]:
%%bash

cd mldl23-ego

PYTHON_PATH=/usr/local/envs/egovision/bin/python
$PYTHON_PATH script_RGB_actionet.py \
  linear \
  /content/mldl23-ego/saved_features/saved_Action_train.pkl \
  0.01 \
  0.9 \
  100

[epoch 1] accuracy: 4.167%
[epoch 2] accuracy: 7.143%
[epoch 3] accuracy: 6.250%
[epoch 4] accuracy: 5.952%
[epoch 5] accuracy: 5.952%
[epoch 6] accuracy: 12.798%
[epoch 7] accuracy: 16.071%
[epoch 8] accuracy: 19.048%
[epoch 9] accuracy: 21.429%
[epoch 10] accuracy: 26.786%
[epoch 11] accuracy: 31.250%
[epoch 12] accuracy: 31.548%
[epoch 13] accuracy: 39.286%
[epoch 14] accuracy: 37.500%
[epoch 15] accuracy: 44.345%
[epoch 16] accuracy: 40.476%
[epoch 17] accuracy: 43.155%
[epoch 18] accuracy: 49.405%
[epoch 19] accuracy: 53.869%
[epoch 20] accuracy: 52.083%
[epoch 21] accuracy: 54.464%
[epoch 22] accuracy: 56.250%
[epoch 23] accuracy: 56.250%
[epoch 24] accuracy: 62.798%
[epoch 25] accuracy: 63.690%
[epoch 26] accuracy: 67.560%
[epoch 27] accuracy: 60.417%
[epoch 28] accuracy: 64.583%
[epoch 29] accuracy: 64.881%
[epoch 30] accuracy: 66.369%
[epoch 31] accuracy: 63.393%
[epoch 32] accuracy: 68.452%
[epoch 33] accuracy: 60.714%
[epoch 34] accuracy: 67.857%
[epoch 35] accuracy: 68.750%

## ACTIONET SPECTROGRAMS CLASSIFIER

In [ ]:
%%bash

cd mldl23-ego

PYTHON_PATH=/usr/local/envs/egovision/bin/python
$PYTHON_PATH script_spectrograms.py \
  ./train_val/Action_train.pkl \
  ./saved_features/saved_Action_train.pkl \
  0.01 \
  0.9 \
  100

FOLD: 0
[epoch 1] accuracy: 5.952%
[epoch 2] accuracy: 5.060%
[epoch 3] accuracy: 2.976%
[epoch 4] accuracy: 5.357%
[epoch 5] accuracy: 3.869%
[epoch 6] accuracy: 6.845%
[epoch 7] accuracy: 6.845%
[epoch 8] accuracy: 5.655%
[epoch 9] accuracy: 6.548%
[epoch 10] accuracy: 7.143%
[epoch 11] accuracy: 7.440%
[epoch 12] accuracy: 7.440%
[epoch 13] accuracy: 9.226%
[epoch 14] accuracy: 8.333%
[epoch 15] accuracy: 7.143%
[epoch 16] accuracy: 7.738%
[epoch 17] accuracy: 7.143%
[epoch 18] accuracy: 10.714%
[epoch 19] accuracy: 11.905%
[epoch 20] accuracy: 13.095%
[epoch 21] accuracy: 10.714%
[epoch 22] accuracy: 10.417%
[epoch 23] accuracy: 11.012%
[epoch 24] accuracy: 11.607%
[epoch 25] accuracy: 11.607%
[epoch 26] accuracy: 13.393%
[epoch 27] accuracy: 15.476%
[epoch 28] accuracy: 12.500%
[epoch 29] accuracy: 13.690%
[epoch 30] accuracy: 16.369%
[epoch 31] accuracy: 16.071%
[epoch 32] accuracy: 19.345%
[epoch 33] accuracy: 19.048%
[epoch 34] accuracy: 18.155%
[epoch 35] accuracy: 17.262%
[ep

### SpecNet for Generalization


In [ ]:
%%bash

cd mldl23-ego

PYTHON_PATH=/usr/local/envs/egovision/bin/python
$PYTHON_PATH script_spec_generalization.py \
  ./train_emg_all_sources/all_emg.pkl \
  ./train_val/Action_train.pkl \
  ./saved_features/saved_Action_train.pkl \
  0.01 \
  0.9 \
  100

[epoch 1] accuracy: 9.834%
[epoch 2] accuracy: 15.193%
[epoch 3] accuracy: 19.917%
[epoch 4] accuracy: 24.779%
[epoch 5] accuracy: 28.619%
[epoch 6] accuracy: 30.994%
[epoch 7] accuracy: 34.669%
[epoch 8] accuracy: 36.713%
[epoch 9] accuracy: 40.552%
[epoch 10] accuracy: 43.591%
[epoch 11] accuracy: 46.685%
[epoch 12] accuracy: 50.138%
[epoch 13] accuracy: 51.243%
[epoch 14] accuracy: 53.564%
[epoch 15] accuracy: 55.249%
[epoch 16] accuracy: 55.829%
[epoch 17] accuracy: 58.122%
[epoch 18] accuracy: 58.785%
[epoch 19] accuracy: 61.657%
[epoch 20] accuracy: 62.569%
[epoch 21] accuracy: 63.398%
[epoch 22] accuracy: 64.613%
[epoch 23] accuracy: 65.221%
[epoch 24] accuracy: 65.663%
[epoch 25] accuracy: 65.608%
[epoch 26] accuracy: 66.906%
[epoch 27] accuracy: 65.773%
[epoch 28] accuracy: 67.707%
[epoch 29] accuracy: 66.934%
[epoch 30] accuracy: 69.282%
[epoch 31] accuracy: 69.392%
[epoch 32] accuracy: 68.867%
[epoch 33] accuracy: 70.166%
[epoch 34] accuracy: 70.304%
[epoch 35] accuracy: 69.

## MultiNet

In [ ]:
%%bash

cd mldl23-ego

chmod 755 ./train_val/Action_train.pkl

PYTHON_PATH=/usr/local/envs/egovision/bin/python
$PYTHON_PATH script_multimodal.py \
  ./train_val/Action_train.pkl \
  ./saved_features/saved_Action_train.pkl \
  0.01 \
  0.9 \
  100

FOLD: 0
[epoch 1] accuracy: 8.333%
[epoch 2] accuracy: 20.536%
[epoch 3] accuracy: 37.202%
[epoch 4] accuracy: 43.155%
[epoch 5] accuracy: 52.679%
[epoch 6] accuracy: 52.083%
[epoch 7] accuracy: 56.548%
[epoch 8] accuracy: 50.000%
[epoch 9] accuracy: 57.143%
[epoch 10] accuracy: 58.333%
[epoch 11] accuracy: 59.226%
[epoch 12] accuracy: 62.798%
[epoch 13] accuracy: 59.524%
[epoch 14] accuracy: 61.012%
[epoch 15] accuracy: 62.798%
[epoch 16] accuracy: 61.905%
[epoch 17] accuracy: 63.690%
[epoch 18] accuracy: 60.714%
[epoch 19] accuracy: 68.452%
[epoch 20] accuracy: 62.202%
[epoch 21] accuracy: 66.369%
[epoch 22] accuracy: 68.452%
[epoch 23] accuracy: 72.024%
[epoch 24] accuracy: 69.643%
[epoch 25] accuracy: 65.476%
[epoch 26] accuracy: 67.560%
[epoch 27] accuracy: 73.810%
[epoch 28] accuracy: 72.024%
[epoch 29] accuracy: 73.810%
[epoch 30] accuracy: 73.214%
[epoch 31] accuracy: 75.595%
[epoch 32] accuracy: 71.429%
[epoch 33] accuracy: 79.167%
[epoch 34] accuracy: 73.512%
[epoch 35] accur